In [0]:
checkpoints = spark.sql("""DESCRIBE EXTERNAL LOCATION `checkpoints`""").select('url').collect()[0][0]
bronze = spark.sql("""DESCRIBE EXTERNAL LOCATION `bronze`""").select('url').collect()[0][0]
silver = spark.sql("""DESCRIBE EXTERNAL LOCATION `silver`""").select('url').collect()[0][0]
gold = spark.sql("""DESCRIBE EXTERNAL LOCATION `gold`""").select('url').collect()[0][0]

In [0]:
print(checkpoints)
print(bronze)
print(silver)
print(gold)

In [0]:
df_silverRoads = spark.readStream.table("`dev_catalog`.`silver`.`silver_road`")

In [0]:
df_silverTraffic = spark.readStream.table("`dev_catalog`.`silver`.`silver_traffic`")


#### Creating Vehicle Intensity column

In [0]:

def create_Veh_Intenisty(df):
    from pyspark.sql.functions import col
    print('Creating Veh_Intensity: ',end='')
    df = df.withColumn('Veh_Intensity', col('Mortor_Vehicles_Count')/col('Link_length_km'))
    print('Success')
    return df

In [0]:
df_vehInte= create_Veh_Intenisty(df_silverTraffic)

In [0]:
df_vehInte.printSchema()

In [0]:
df_silverTraffic.printSchema()

#### Creating loadTime Column

In [0]:
from pyspark.sql.functions import current_timestamp
def create_LoadTime(df):
    print('Creating LoadTime: ',end='')
    df = df.withColumn('LoadTime', current_timestamp())
    print('Success')
    return df


In [0]:
df_loadTime =create_LoadTime(df_vehInte)

In [0]:
df_loadTimeRoad = create_LoadTime(df_silverRoads)

#### Write data to Gold Traffic

In [0]:
df_loadTime.writeStream \
    .format("delta") \
    .option("checkpointLocation",f'{checkpoints}/GoldTrafficLoad/Checkpt') \
    .outputMode("append") \
    .trigger(availableNow=True) \
    .toTable("`dev_catalog`.`gold`.`gold_traffic`")

In [0]:
df_loadTimeRoad.writeStream \
    .format("delta") \
    .option("checkpointLocation",f'{checkpoints}/GoldRoadLoad/Checkpt') \
    .outputMode("append") \
    .trigger(availableNow=True) \
    .toTable("`dev_catalog`.`gold`.`gold_road`")